In [1]:
import os
import requests
from pyairtable import Api
import pandas as pd
import numpy as np
import xlwings as xw
import diccionarios as dn
import directorios as dir
import funciones as fn

In [3]:
from datetime import datetime, date, timedelta

today = date(2024, 9, 30)

In [4]:
alertas_cols = [
    "POZO", "APERTURA", "ETAPA", "CIERRE", "TIPO DE ALERTA", "NIVEL DE ALERTA", "ATENCION", "ACTIVIDAD", "ALERTA EMITIDA",
    "RESUMEN DE LA ALERTA", "DESCRIPCION", "CAUSA", "DETALLE", "INFORMADO", "PERSONAL QUE ATIENDE", "COMENTARIO", "AREA DE OPORTUNIDAD"
]

tnp_cols = [
    'POZO', 'ETAPA', 'APERTURA', 'TIEMPO (Dias)', 'SUB-CLASIFICACION', 'CONCEPTO', 'CLASIFICACION'
]

tnpi_cols = ["POZO", "ETAPA", "FECHA", "TNPI GENERADO", "OPERACION", "SUB-OPERACION", "CAUSA", "DETALLE", "AREA DE OPORTUNIDAD"
]

In [5]:
alertas_api_key = os.getenv('AIRTABLE_API_KEY', 'patk6ag0UsQrjK1nn.e5e08f87ff10103893fafa40d8f0210a50cd63dfb93255b8abaf258de292de89')
alertas_base_id = 'appTA2GBh37ydQWuR'
main_table_name = 'Monitoreo de alertas'
alertas_table_id = 'tblF0z5i5fu0ApizJ'
linked_table_id = 'tblARVQJHR6wNfeS0' 

api = Api(alertas_api_key)
alertas_table = api.table(alertas_base_id, alertas_table_id)

try:
    records = alertas_table.all()
    print("Records retrieved successfully")

    alertas = pd.DataFrame([record['fields'] for record in records])

    if 'APERTURA' in alertas.columns:
        alertas['APERTURA'] = pd.to_datetime(alertas['APERTURA'], errors='coerce')
        if alertas['APERTURA'].dt.tz is None:
            alertas['APERTURA'] = alertas['APERTURA'].dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City')
        else:
            alertas['APERTURA'] = alertas['APERTURA'].dt.tz_convert('America/Mexico_City')

    if 'CIERRE' in alertas.columns:
        alertas['CIERRE'] = pd.to_datetime(alertas['CIERRE'], errors='coerce')
        if alertas['CIERRE'].dt.tz is None:
            alertas['CIERRE'] = alertas['CIERRE'].dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City')
        else:
            alertas['CIERRE'] = alertas['CIERRE'].dt.tz_convert('America/Mexico_City')

    if alertas['APERTURA'].isna().any() or alertas['CIERRE'].isna().any():
        print("Warning: Some datetime conversions failed and resulted in NaT.")

    headers = {
    'Authorization': f'Bearer {alertas_api_key}',
    }

    response = requests.get(f'https://api.airtable.com/v0/{alertas_base_id}/{alertas_table_id}', headers=headers)
    data = response.json()

    linked_table_response = requests.get(f'https://api.airtable.com/v0/{alertas_base_id}/{linked_table_id}', headers=headers)
    linked_table_data = linked_table_response.json()

    linked_map = {record['id']: record['fields']['POZO'] for record in linked_table_data['records']}

    for record in data['records']:
        linked_id = record['fields'].get('POZO')
        if linked_id:
            record['fields']['POZO'] = linked_map.get(linked_id[0], 'Unknown')

    alertas['POZO'] = alertas['POZO'].apply(lambda x: x[0] if isinstance(x, list) and x else x)
    alertas['POZO'] = alertas['POZO'].map(linked_map).fillna('Unknown')
    alertas = alertas.sort_values(by=['POZO', 'ETAPA', 'APERTURA']).reset_index()
    alertas = alertas[alertas_cols]

    # one_drive = os.getenv("OneDrive")
    # directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "1. Alertas", "Todas las ALERTAS.xlsx")
    # directorio_de_salida = os.path.normpath(directorio_de_salida)

    with xw.App(visible=False) as app:
        wb = xw.Book()
        ws = wb.sheets[0]
        ws.name = 'ALERTAS'
        ws.range('A1').options(index=False).value = alertas
        wb.save(dir.alerts_path_file)
        wb.close()
    
    print("Data saved to Excel successfully using xlwings")

except Exception as e:
    print("Failed to retrieve or save records:", str(e))


Records retrieved successfully
Data saved to Excel successfully using xlwings


In [6]:
tnp_api_key = os.getenv('AIRTABLE_API_KEY', 'patk6ag0UsQrjK1nn.e5e08f87ff10103893fafa40d8f0210a50cd63dfb93255b8abaf258de292de89')
tnp_base_id = 'appTA2GBh37ydQWuR'
tnp_table_id = 'tblOYu31lRs5zLSxR'

api = Api(tnp_api_key)
tnp_table = api.table(tnp_base_id, tnp_table_id)

try:
    records = tnp_table.all()
    print("Records retrieved successfully")

    tnp = pd.DataFrame([record['fields'] for record in records])

    tnp['APERTURA'] = pd.to_datetime(tnp['APERTURA'], format='%Y-%m-%d')

    tnp = tnp.sort_values(by=['POZO', 'ETAPA', 'APERTURA']).reset_index()
    tnp = tnp[tnp_cols]

    # one_drive = os.getenv("OneDrive")
    # directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "2. Tiempo no productivo", "Todos los TNP.xlsx")
    # directorio_de_salida = os.path.normpath(directorio_de_salida)

    tnp.to_excel(dir.npt_path_file, sheet_name='TNP', index=False)
    print("Data saved to Excel successfully")

except Exception as e:
    print("Failed to retrieve or save records:", str(e))

Records retrieved successfully
Data saved to Excel successfully


In [7]:
tnpi_api_key = os.getenv('AIRTABLE_API_KEY', 'patk6ag0UsQrjK1nn.e5e08f87ff10103893fafa40d8f0210a50cd63dfb93255b8abaf258de292de89')
tnpi_base_id = 'appTA2GBh37ydQWuR'
tnpi_table_id = 'tblRHiTnYaXL7QVjJ'

api = Api(tnpi_api_key)
tnpi_table = api.table(tnpi_base_id, tnpi_table_id)

try:
    records = tnpi_table.all()
    print("Records retrieved successfully")

    tnpi = pd.DataFrame([record['fields'] for record in records])
    tnpi = tnpi.drop(columns=['Creado por'], errors='ignore')

    tnpi['FECHA'] = pd.to_datetime(tnpi['FECHA'], format='%Y-%m-%d')
    tnpi_columns = {'SUBOPERACION':'SUB-OPERACION'}
    tnpi.rename(columns=tnpi_columns, inplace=True)

    tnpi = tnpi.sort_values(by=['POZO', 'ETAPA', 'FECHA']).reset_index()
    tnpi = tnpi[tnpi_cols]

    # one_drive = os.getenv("OneDrive")
    # directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "3. Tiempo no productivo invisible", "Todos los TNPI.xlsx")
    # directorio_de_salida = os.path.normpath(directorio_de_salida)

    tnpi.to_excel(dir.inpt_path_file, sheet_name='TNPI', index=False)
    print("Data saved to Excel successfully")

except Exception as e:
    print("Failed to retrieve or save records:", str(e))

Records retrieved successfully
Data saved to Excel successfully


#### Tabla de corte diario de TNP / TNPI

In [8]:
alertas_hoy = alertas[alertas['APERTURA'].dt.date == today]
tnp_hoy = tnp[tnp['APERTURA'].dt.date == today]
tnpi_hoy = tnpi[tnpi['FECHA'].dt.date == today]

In [9]:
alertas_hoy.loc[:, 'APERTURA'] = alertas_hoy['APERTURA'].dt.tz_localize(None)
alertas_hoy.loc[:, 'CIERRE'] = alertas_hoy['CIERRE'].dt.tz_localize(None)
one_drive = os.getenv("OneDrive")
directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "1. Alertas", "ALERTAS.xlsx")
directorio_de_salida = os.path.normpath(directorio_de_salida)
alertas_hoy.to_excel(directorio_de_salida, sheet_name="ALERTAS", index=False, engine='xlsxwriter')

C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\1070307999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertas_hoy.loc[:, 'APERTURA'] = alertas_hoy['APERTURA'].dt.tz_localize(None)
C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\1070307999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertas_hoy.loc[:, 'CIERRE'] = alertas_hoy['CIERRE'].dt.tz_localize(None)


In [10]:
tnp_hoy.loc[:, 'APERTURA'] = tnp_hoy['APERTURA'].dt.tz_localize(None)
one_drive = os.getenv("OneDrive")
directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "2. Tiempo no productivo", "TNP.xlsx")
directorio_de_salida = os.path.normpath(directorio_de_salida)
tnp_hoy.to_excel(directorio_de_salida, sheet_name='TNP', index=False)

C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\3736368351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnp_hoy.loc[:, 'APERTURA'] = tnp_hoy['APERTURA'].dt.tz_localize(None)
C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\3736368351.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tnp_hoy.loc[:, 'APERTURA'] = tnp_hoy['APERTURA'].dt.tz_localize(None)


In [11]:
tnpi_hoy.loc[:, 'FECHA'] = tnpi_hoy['FECHA'].dt.tz_localize(None)
one_drive = os.getenv("OneDrive")
directorio_de_salida = os.path.join(one_drive, "CNT", "2. Data", "3. Tablas (alertas_tnp_tnpi)", "3. Tiempo no productivo invisible", "TNPI.xlsx")
directorio_de_salida = os.path.normpath(directorio_de_salida)
tnpi_hoy.to_excel(directorio_de_salida, sheet_name='TNPI', index=False)

C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\1062570567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tnpi_hoy.loc[:, 'FECHA'] = tnpi_hoy['FECHA'].dt.tz_localize(None)
C:\Users\VictorTorres\AppData\Local\Temp\ipykernel_67960\1062570567.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tnpi_hoy.loc[:, 'FECHA'] = tnpi_hoy['FECHA'].dt.tz_localize(None)


#### Calculo de horas de TNPI diario y cumulativo

In [12]:
tnpi_totales = tnpi[['POZO', 'FECHA', 'TNPI GENERADO']].groupby(['POZO', 'FECHA']).sum().reset_index()

# Add the cumulative sum for each POZO
tnpi_totales['TNPI CUMULATIVO'] = tnpi_totales.groupby('POZO')['TNPI GENERADO'].cumsum()

tnpi_totales = tnpi_totales.rename(columns={'TNPI GENERADO': 'TNPI 24h', 
                                    'TNPI CUMULATIVO': 'TNPI total'})
tnpi_totales

,POZO,FECHA,TNPI 24h,TNPI total
0,ACTUL 3,2024-03-30,1.40,1.40
1,ACTUL 3,2024-03-31,0.64,2.04
2,ACTUL 3,2024-04-01,1.23,3.27
3,ACTUL 3,2024-04-05,0.78,4.05
4,ACTUL 3,2024-04-06,0.44,4.49
...,...,...,...,...
4518,XANAB 58A,2024-09-17,0.06,14.51
4519,XANAB 58A,2024-09-20,0.32,14.83
4520,XANAB 58A,2024-09-21,0.08,14.91
4521,XANAB 58A,2024-09-22,0.45,15.36


#### Calculo de horas de TNP diario y cumulativo

In [13]:
tnp_totales = tnp.copy()
tnp_totales = tnp_totales.rename(
    columns = {
        "TIEMPO (Dias)": "TNP GENERADO",
        'APERTURA': 'FECHA'
        })
tnp_totales['TNP GENERADO'] = tnp_totales['TNP GENERADO'] * 24

In [14]:
tnp_totales = tnp_totales[['POZO', 'FECHA', 'TNP GENERADO']].groupby(['POZO', 'FECHA']).sum().reset_index()

tnp_totales['TNP CUMULATIVO'] = tnp_totales.groupby('POZO')['TNP GENERADO'].cumsum()

tnp_totales = tnp_totales.rename(columns={'TNP GENERADO': 'TNP 24h', 
                                    'TNP CUMULATIVO': 'TNP total'})
tnp_totales

,POZO,FECHA,TNP 24h,TNP total
0,ACTUL 4,2024-06-15,6.72,6.72
1,ACTUL 4,2024-06-16,16.80,23.52
2,ACTUL 4,2024-06-17,5.28,28.80
3,ACTUL 4,2024-06-20,4.08,32.88
4,ACTUL 4,2024-06-21,2.16,35.04
...,...,...,...,...
3698,XANAB 58A,2024-09-23,0.72,129.12
3699,XANAB 58A,2024-09-24,0.48,129.60
3700,XANAB 58A,2024-09-25,24.24,153.84
3701,XANAB 58A,2024-09-26,24.00,177.84


In [15]:
tiempos_diarios = pd.merge(tnp_totales, tnpi_totales, on=['POZO', 'FECHA'], how='outer')

In [16]:
for col in ['TNP 24h', 'TNPI 24h', 'TNP total', 'TNPI total']:
    tiempos_diarios[col] = tiempos_diarios[col].replace(np.nan, 0)

tiempos_diarios['TP'] = 24 - tiempos_diarios['TNPI 24h'] - tiempos_diarios['TNP 24h']
tiempos_diarios['TP'] = tiempos_diarios['TP'].replace(np.nan, 24)
tiempos_diarios = tiempos_diarios[['POZO', 'FECHA', 'TP', 'TNPI 24h', 'TNP 24h', 'TNPI total', 'TNP total']]
tiempos_diarios

,POZO,FECHA,TP,TNPI 24h,TNP 24h,TNPI total,TNP total
0,ACTUL 4,2024-06-15,16.40,0.88,6.72,3.56,6.72
1,ACTUL 4,2024-06-16,7.20,0.00,16.80,0.00,23.52
2,ACTUL 4,2024-06-17,17.70,1.02,5.28,4.58,28.80
3,ACTUL 4,2024-06-20,19.92,0.00,4.08,0.00,32.88
4,ACTUL 4,2024-06-21,20.70,1.14,2.16,6.31,35.04
...,...,...,...,...,...,...,...
6542,XANAB 58A,2024-09-07,23.88,0.12,0.00,12.58,0.00
6543,XANAB 58A,2024-09-11,22.70,1.30,0.00,13.88,0.00
6544,XANAB 58A,2024-09-15,23.43,0.57,0.00,14.45,0.00
6545,XANAB 58A,2024-09-22,23.55,0.45,0.00,15.36,0.00


In [17]:
directorio_de_salida = os.path.join(dir.data_folder_path, "daily_sum.xlsx")
directorio_de_salida = os.path.normpath(directorio_de_salida)
tiempos_diarios.to_excel(directorio_de_salida, sheet_name='CorteTNPI', index=False)